In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
from IPython.display import display, Javascript

disable_js = """
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
"""

def load_ipython_extension(ip):
    display(Javascript(disable_js))
    print ("autoscrolling long output is disabled")
load_ipython_extension("")


from IPython.display import display_html
from itertools import chain,cycle
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2 style="text-align: center;">{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

<IPython.core.display.Javascript object>

autoscrolling long output is disabled


In [3]:
repo_path = os.path.abspath(os.path.join('../..'))
display(os.listdir(repo_path))

sys.path.insert(0, repo_path)

['.git',
 '.idea',
 'bet_races_notes',
 'data',
 'f1pm',
 'ModelOne_TODOsNotes',
 'notebooks']

In [4]:
from f1pm.historicaldataprocessing.historical_data_processing_m1 import process_historical_historical_data_m1
from f1pm.historicaldataprocessing.tools import compute_historical_sub_data_set
from f1pm.probabilityestimates.pe_historical_data import ProbabilityEstimateHistoricalData
from f1pm.betstrategiesevaluations.model_strategy_zero import ModelOneStrategyZero
from f1pm.webrequests.f1com_standing_requests import request_current_drivers_standing, request_quali_results, request_current_constructors_standing

# Historical data processing

In [5]:
df_data, df_data_all = process_historical_historical_data_m1()
display(df_data)

,raceId,race_name,driverRef,qualifying_position,grid,position,positionOrder,driver_standing_position,constructor_standing_position,round,year,url
0,18,Australian Grand Prix - 2008,hamilton,1.0,1,1,1,1.0,1.0,1,2008.0,http://en.wikipedia.org/wiki/2008_Australian_G...
1,18,Australian Grand Prix - 2008,heidfeld,5.0,5,2,2,2.0,3.0,1,2008.0,http://en.wikipedia.org/wiki/2008_Australian_G...
2,18,Australian Grand Prix - 2008,rosberg,7.0,7,3,3,3.0,2.0,1,2008.0,http://en.wikipedia.org/wiki/2008_Australian_G...
3,18,Australian Grand Prix - 2008,alonso,12.0,11,4,4,4.0,4.0,1,2008.0,http://en.wikipedia.org/wiki/2008_Australian_G...
4,18,Australian Grand Prix - 2008,kovalainen,3.0,3,5,5,5.0,1.0,1,2008.0,http://en.wikipedia.org/wiki/2008_Australian_G...
...,...,...,...,...,...,...,...,...,...,...,...,...
25715,1089,Italian Grand Prix - 2022,kevin_magnussen,19.0,16,16,16,12.0,7.0,16,2022.0,http://en.wikipedia.org/wiki/2022_Italian_Gran...
25716,1089,Italian Grand Prix - 2022,ricciardo,8.0,4,\N,17,14.0,5.0,16,2022.0,http://en.wikipedia.org/wiki/2022_Italian_Gran...
25717,1089,Italian Grand Prix - 2022,stroll,18.0,12,\N,18,18.0,9.0,16,2022.0,http://en.wikipedia.org/wiki/2022_Italian_Gran...
25718,1089,Italian Grand Prix - 2022,alonso,10.0,6,\N,19,9.0,4.0,16,2022.0,http://en.wikipedia.org/wiki/2022_Italian_Gran...


## Loading Current driver championship standings

In [6]:
driver_standing_url = "https://www.formula1.com/en/results.html/2022/drivers.html"
current_driver_standing_table = request_current_drivers_standing(driver_standing_url)
current_driver_standing_table = current_driver_standing_table.set_index('DRIVER')
display(current_driver_standing_table)

,POS,NATIONALITY,CAR,PTS
DRIVER,,,,
Max Verstappen VER,1,NED,Red Bull Racing RBPT,366
Sergio Perez PER,2,MEX,Red Bull Racing RBPT,253
Charles Leclerc LEC,3,MON,Ferrari,252
George Russell RUS,4,GBR,Mercedes,207
Carlos Sainz SAI,5,ESP,Ferrari,202
Lewis Hamilton HAM,6,GBR,Mercedes,180
Lando Norris NOR,7,GBR,McLaren Mercedes,101
Esteban Ocon OCO,8,FRA,Alpine Renault,78
Fernando Alonso ALO,9,ESP,Alpine Renault,65


In [7]:
### 'manual' updates in current driver championship standings
# current_driver_standing_table.loc['Sergio Perez PER', 'POS'] = 3
# current_driver_standing_table.loc['Charles Leclerc LEC', 'POS'] = 2

display(current_driver_standing_table)

,POS,NATIONALITY,CAR,PTS
DRIVER,,,,
Max Verstappen VER,1,NED,Red Bull Racing RBPT,366
Sergio Perez PER,2,MEX,Red Bull Racing RBPT,253
Charles Leclerc LEC,3,MON,Ferrari,252
George Russell RUS,4,GBR,Mercedes,207
Carlos Sainz SAI,5,ESP,Ferrari,202
Lewis Hamilton HAM,6,GBR,Mercedes,180
Lando Norris NOR,7,GBR,McLaren Mercedes,101
Esteban Ocon OCO,8,FRA,Alpine Renault,78
Fernando Alonso ALO,9,ESP,Alpine Renault,65


## Loading Current constructor championship standings

In [8]:
constructor_standing_url = "https://www.formula1.com/en/results.html/2022/team.html"
current_constructors_standing_table = request_current_constructors_standing(constructor_standing_url)
display(current_constructors_standing_table)

,POS,TEAM,PTS
0,1,Red Bull Racing RBPT,619
1,2,Ferrari,454
2,3,Mercedes,387
3,4,Alpine Renault,143
4,5,McLaren Mercedes,130
5,6,Alfa Romeo Ferrari,52
6,7,Aston Martin Aramco Mercedes,45
7,8,Haas Ferrari,34
8,9,AlphaTauri RBPT,34
9,10,Williams Mercedes,8


## Loading Race grid results

In [9]:
quali_results_url = "https://www.formula1.com/en/results.html/2022/races/1134/japan/qualifying.html"
quali_results_url = "https://www.formula1.com/en/results.html/2022/races/1133/singapore/qualifying.html"
grid_results = request_quali_results(quali_results_url)
grid_results = grid_results.set_index('DRIVER')
display(grid_results)

,POS,NO,CAR,Q1,Q2,Q3,LAPS
DRIVER,,,,,,,
Charles Leclerc LEC,1,16,Ferrari,1:54.129,1:52.343,1:49.412,20
Sergio Perez PER,2,11,Red Bull Racing RBPT,1:54.404,1:52.818,1:49.434,20
Lewis Hamilton HAM,3,44,Mercedes,1:53.161,1:52.691,1:49.466,24
Carlos Sainz SAI,4,55,Ferrari,1:54.559,1:53.219,1:49.583,22
Fernando Alonso ALO,5,14,Alpine Renault,1:55.360,1:53.127,1:49.966,23
Lando Norris NOR,6,4,McLaren Mercedes,1:55.914,1:53.942,1:50.584,24
Pierre Gasly GAS,7,10,AlphaTauri RBPT,1:55.606,1:53.546,1:51.211,24
Max Verstappen VER,8,1,Red Bull Racing RBPT,1:53.057,1:52.723,1:51.395,20
Kevin Magnussen MAG,9,20,Haas Ferrari,1:55.103,1:54.006,1:51.573,23


## Processing Race day grid df

In [10]:
df_race_day_grid = ModelOneStrategyZero.compute_race_day_grid_df(current_driver_standing_table, grid_results, current_constructors_standing_table)
display(df_race_day_grid)

,grid,driver_championship_standing,constructor_championship_standing
DRIVER,,,
Charles Leclerc LEC,1,3,2
Sergio Perez PER,2,2,1
Lewis Hamilton HAM,3,6,3
Carlos Sainz SAI,4,5,2
Fernando Alonso ALO,5,9,4
Lando Norris NOR,6,7,5
Pierre Gasly GAS,7,13,9
Max Verstappen VER,8,1,1
Kevin Magnussen MAG,9,14,8


# Strategy zero evaluation

## Grid estimates

In [11]:
from ipywidgets import Layout, Dropdown, interact, SelectMultiple, Checkbox

target_cumsum_position_dp_ge = Dropdown(options=list(range(1, 22)),
                                     description='target position or best',
                                     style={'description_width': 'initial'})

ci_dp_ge = Dropdown(options=[0.05, 0.10, 0.20, 0.32, 0.40, 0.5], description='ci', style={'description_width': 'initial'})

data_set_year_lower_threshold_dp_ge = Dropdown(options=[2000, 2004, 2008, 2012, 2016, None],
                                            description='year lower threshold',
                                           style={'description_width': 'initial'})

data_set_year_upper_threshold_dp_ge = Dropdown(options=[None, 2000, 2004, 2008, 2012, 2016, 2022],
                                            description='year upper threshold',
                                           style={'description_width': 'initial'})

data_set_round_lower_threshold_dp_ge = Dropdown(options=[5, 10, 15, None],
                                            description='round lower threshold',
                                            style={'description_width': 'initial'})

data_set_round_upper_threshold_dp_ge = Dropdown(options=[None, 10, 15],
                                            description='round upper threshold',
                                            style={'description_width': 'initial'})

look_for_constructor_standing_dp_ge = Checkbox(
    value=False,
    description='look for constructor standing',
    disabled=False,
    indent=False
)


# driver_championship_standing = HBox([Label('Driver Championship standing'), Dropdown(options=list(range(1, 22)))])



In [12]:
def compute_grid_estimate_fuct(target_cumsum_position, ci,
                                                    year_lower_threshold, year_upper_threshold,
                                                    round_lower_threshold, round_upper_threshold, 
                               look_for_constructor_standing):
    df_driver_standing = df_race_day_grid[['driver_championship_standing', 'constructor_championship_standing']]
    df_driver_standing = df_driver_standing.sort_values(by=['driver_championship_standing'])

    
    # Probability estimates object ===================================================================================
    subdatset_params_dict = dict(year_lower_threshold=year_lower_threshold, year_upper_threshold=year_upper_threshold,
                                 round_lower_threshold=round_lower_threshold, round_upper_threshold=round_upper_threshold)

    print(subdatset_params_dict)
    pehd = ProbabilityEstimateHistoricalData(df_data, subdatset_params_dict)
    # ================================================================================================================
    
    # Strategy zero evaluation =======================================================================================
    mosz = ModelOneStrategyZero(pehd)
    sz_grid_prob_estimate = mosz.compute_grid_estimate(target_cumsum_position=target_cumsum_position,
                                                       current_driver_standing_table=df_driver_standing,
                                                       ci=ci,
                                                       subset_n_threshold=10,
                                                       look_for_constructor_standing=look_for_constructor_standing)
    # ================================================================================================================
    
    # Outputs prints ---------------------------------------------
    print("=="*50)
    
    print("Inputs: \n", subdatset_params_dict, 
          "\n target_cumsum_position: ", target_cumsum_position,
         "\n ci: ", ci)
    
    print("=="*50)
    
    # Rename of columns (dor better display)
    race_target_position_prob = sz_grid_prob_estimate.race_target_position_prob.rename(columns={'driver_championship_standing': 'champ. stand.',
                                                                                               'constructor_championship_standing': 'const stand.'})
    fair_bet_value = sz_grid_prob_estimate.fair_bet_value.rename(columns={'driver_championship_standing': 'champ. stand.',
                                                                         'constructor_championship_standing': 'const stand.'})
    
    display_side_by_side(race_target_position_prob, 
                         fair_bet_value, 
                         titles=['grid target position prob', 'fair bet value'])
    
    

In [13]:
_ = interact(compute_grid_estimate_fuct, 
             target_cumsum_position=target_cumsum_position_dp_ge,
             ci=ci_dp_ge,
             year_lower_threshold=data_set_year_lower_threshold_dp_ge, 
             year_upper_threshold=data_set_year_upper_threshold_dp_ge,
             round_lower_threshold=data_set_round_lower_threshold_dp_ge, 
             round_upper_threshold=data_set_round_upper_threshold_dp_ge,
            look_for_constructor_standing=look_for_constructor_standing_dp_ge)

interactive(children=(Dropdown(description='target position or best', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, …

## Race Estimates

In [14]:
from ipywidgets import Layout, Dropdown, interact, SelectMultiple, Checkbox

target_cumsum_position_dp = Dropdown(options=list(range(1, 22)),
                                     description='target position or best',
                                     style={'description_width': 'initial'})

ci_dp = Dropdown(options=[0.05, 0.10, 0.20, 0.32, 0.40, 0.5], description='ci', style={'description_width': 'initial'})

data_set_year_lower_threshold_dp = Dropdown(options=[2000, 2004, 2008, 2012, 2016, None],
                                            description='year lower threshold',
                                           style={'description_width': 'initial'})

data_set_year_upper_threshold_dp = Dropdown(options=[None, 2000, 2004, 2008, 2012, 2016, 2022],
                                            description='year upper threshold',
                                           style={'description_width': 'initial'})

data_set_round_lower_threshold_dp = Dropdown(options=[5, 10, 15, None],
                                            description='round lower threshold',
                                            style={'description_width': 'initial'})

data_set_round_upper_threshold_dp = Dropdown(options=[None, 10, 15],
                                            description='round upper threshold',
                                            style={'description_width': 'initial'})

look_for_constructor_standing_dp = Checkbox(
    value=False,
    description='look for constructor standing',
    disabled=False,
    indent=False
)
# driver_championship_standing = HBox([Label('Driver Championship standing'), Dropdown(options=list(range(1, 22)))])



In [15]:
def compute_race_estimate_fuct(target_cumsum_position, ci,
                                                    year_lower_threshold, year_upper_threshold,
                                                    round_lower_threshold, round_upper_threshold,
                              look_for_constructor_standing):
    df_driver_standing = df_race_day_grid[['driver_championship_standing', 'constructor_championship_standing']]
    df_driver_standing = df_driver_standing.sort_values(by=['driver_championship_standing'])

    
    # Probability estimates object ===================================================================================
    subdatset_params_dict = dict(year_lower_threshold=year_lower_threshold, year_upper_threshold=year_upper_threshold,
                                 round_lower_threshold=round_lower_threshold, round_upper_threshold=round_upper_threshold)

    print(subdatset_params_dict)
    pehd = ProbabilityEstimateHistoricalData(df_data, subdatset_params_dict)
    # ================================================================================================================
    
    # Strategy zero evaluation =======================================================================================
    mosz = ModelOneStrategyZero(pehd)
    sz_race_prob_estimate = mosz.compute_race_estimate(target_cumsum_position=target_cumsum_position,
                                                       current_driver_standing_table=df_driver_standing,
                                                       ci=ci,
                                                       subset_n_threshold=10,
                                                      look_for_constructor_standing=look_for_constructor_standing)
    # ================================================================================================================
    
    # Outputs prints ---------------------------------------------
    print("=="*50)
    
    print("Inputs: \n", subdatset_params_dict, 
          "\n target_cumsum_position: ", target_cumsum_position,
         "\n ci: ", ci)
    
    print("=="*50)
    
    # Rename of columns (dor better display)
    race_target_position_prob = sz_race_prob_estimate.race_target_position_prob.rename(columns={'driver_championship_standing': 'champ. stand.',
                                                                                               'constructor_championship_standing': 'const stand.'})
    fair_bet_value = sz_race_prob_estimate.fair_bet_value.rename(columns={'driver_championship_standing': 'champ. stand.',
                                                                         'constructor_championship_standing': 'const stand.'})
    
    display_side_by_side(race_target_position_prob, 
                         fair_bet_value, 
                         titles=['race target position prob', 'fair bet value'])
    
    

In [16]:
_ = interact(compute_race_estimate_fuct, 
             target_cumsum_position=target_cumsum_position_dp,
             ci=ci_dp,
             year_lower_threshold=data_set_year_lower_threshold_dp, 
             year_upper_threshold=data_set_year_upper_threshold_dp,
             round_lower_threshold=data_set_round_lower_threshold_dp, 
             round_upper_threshold=data_set_round_upper_threshold_dp,
            look_for_constructor_standing=look_for_constructor_standing_dp)

interactive(children=(Dropdown(description='target position or best', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, …

## Race Estimates conditioning of grid

In [17]:
from ipywidgets import Layout, Dropdown, interact, SelectMultiple, Checkbox

target_cumsum_position_dp_cog = Dropdown(options=list(range(1, 22)),
                                         description='target position or best',
                                         style={'description_width': 'initial'})

ci_dp_cog = Dropdown(options=[0.05, 0.10, 0.20, 0.32, 0.40, 0.5], description='ci', style={'description_width': 'initial'})

data_set_year_lower_threshold_dp_cog = Dropdown(options=[2000, 2004, 2008, 2012, 2016, None],
                                            description='year lower threshold',
                                           style={'description_width': 'initial'})

data_set_year_upper_threshold_dp_cog = Dropdown(options=[None, 2000, 2004, 2008, 2012, 2016, 2022],
                                            description='year upper threshold',
                                           style={'description_width': 'initial'})

data_set_round_lower_threshold_dp_cog = Dropdown(options=[5, 10, 15, None],
                                            description='round lower threshold',
                                            style={'description_width': 'initial'})

data_set_round_upper_threshold_dp_cog = Dropdown(options=[None, 10, 15],
                                            description='round upper threshold',
                                            style={'description_width': 'initial'})

look_for_constructor_standing_dp_cog = Checkbox(
    value=False,
    description='look for constructor standing',
    disabled=False,
    indent=False
)

# driver_championship_standing = HBox([Label('Driver Championship standing'), Dropdown(options=list(range(1, 22)))])



In [18]:
def compute_conditioning_on_grid_race_estimate_fuct(target_cumsum_position, ci,
                                                    year_lower_threshold, year_upper_threshold,
                                                    round_lower_threshold, round_upper_threshold,
                                                   look_for_constructor_standing):
    
    # Probability estimates object ===================================================================================
    subdatset_params_dict = dict(year_lower_threshold=year_lower_threshold, year_upper_threshold=year_upper_threshold,
                                 round_lower_threshold=round_lower_threshold, round_upper_threshold=round_upper_threshold)

    pehd = ProbabilityEstimateHistoricalData(df_data, subdatset_params_dict)
    # ================================================================================================================
    
    # Strategy zero evaluation =======================================================================================
    mosz = ModelOneStrategyZero(pehd)
    sz_race_prob_estimate = mosz.compute_conditioning_on_grid_race_estimate(target_cumsum_position=target_cumsum_position,
                                                                                 race_day_grid=df_race_day_grid,
                                                                                 ci=ci,
                                                                                 subset_n_threshold=10,
                                                                           look_for_constructor_standing=look_for_constructor_standing)
    # ================================================================================================================
    
    # Outputs prints ---------------------------------------------
    print("=="*50)
    
    print("Inputs: \n", subdatset_params_dict, 
          "\n target_cumsum_position: ", target_cumsum_position,
         "\n ci: ", ci)
    
    print("=="*50)
    
    # Rename of columns (dor better display)
    race_target_position_prob = sz_race_prob_estimate.race_target_position_prob.rename(columns={'driver_championship_standing': 'champ. stand.',
                                                                                               'constructor_championship_standing': 'const stand.'})
    fair_bet_value = sz_race_prob_estimate.fair_bet_value.rename(columns={'driver_championship_standing': 'champ. stand.',
                                                                         'constructor_championship_standing': 'const stand.'})
    
    display_side_by_side(race_target_position_prob, 
                         fair_bet_value, 
                         titles=['race target position prob', 'fair bet value'])
    
    if target_cumsum_position == 1:
        race_target_position_prob_normalized = sz_race_prob_estimate.race_target_position_prob_normalized.rename(columns={'driver_championship_standing': 'champ. stand.',
                                                                                                                         'constructor_championship_standing': 'const stand.'})
        fair_bet_value_normalized = sz_race_prob_estimate.fair_bet_value_normalized.rename(columns={'driver_championship_standing': 'champ. stand.',
                                                                                                   'constructor_championship_standing': 'const stand.'})

        display_side_by_side(race_target_position_prob_normalized, 
                         fair_bet_value_normalized, 
                         titles=['race target position prob normalized', 'fair bet value normalized'])
    
    

In [19]:
_ = interact(compute_conditioning_on_grid_race_estimate_fuct, 
             target_cumsum_position=target_cumsum_position_dp_cog,
             ci=ci_dp_cog,
             year_lower_threshold=data_set_year_lower_threshold_dp_cog, 
             year_upper_threshold=data_set_year_upper_threshold_dp_cog,
             round_lower_threshold=data_set_round_lower_threshold_dp_cog, 
             round_upper_threshold=data_set_round_upper_threshold_dp_cog,
            look_for_constructor_standing=look_for_constructor_standing_dp_cog)

interactive(children=(Dropdown(description='target position or best', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, …